8. Reinf learning. Exercise: Use policy gradients to solve OpenAI Gym's LunarLander-v2 environment. You will need to install the Box2D dependencies (%pip install -U gym[box2d]).

In [2]:
import gym
env = gym.make("LunarLander-v2")
obs = env.reset()